<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/seperatemodeling_gwn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [ ]:
import pandas as pd
import numpy as np

from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import make_scorer
from ddop.metrics import average_costs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading data-set

*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## Split in feature and target matrix

In [ ]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split

In [ ]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## Definition of the item_ids 


*   will be needed within the for loops later
*   will contain all 25 products

In [ ]:
prods = list(range(0,25))

## Determining some parameters
*   the potential params and their values for randomized search are defined
*   the estimator, the scorer and finally the whole grid is defined

In [ ]:
kernel_bandwidth = np.array([20, 25, 28, 30, 35, 40, 50, 60, 80, 100])
param_random = dict(kernel_bandwidth = kernel_bandwidth)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# gwn for 95% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,95
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_95 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_95)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_95.append(kbwth)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_gwn_95_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  
  knn = GaussianWeightedNewsvendor(cu = cu, co = co_95, kernel_bandwidth = best_param_kbwth_95[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_95),2)
  result_gwn_95_avc.append(avc)

In [ ]:
GWN_separate_avc_95 = pd.DataFrame(result_gwn_95_avc, columns=['GWN_avc_95'])
GWN_separate_avc_95.index.name = 'item_id'

In [ ]:
result_gwn_95_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_95, kernel_bandwidth = best_param_kbwth_95[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_95)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_95)
  result_gwn_95_pscr.append(pscr)

In [ ]:
GWN_separate_pscr_95 = pd.DataFrame(result_gwn_95_pscr, columns=['GWN_pscr_95'])
GWN_separate_pscr_95.index.name = 'item_id'

# gwn for 90% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,9
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_90 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_90.append(kbwth)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_gwn_90_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  
  knn = GaussianWeightedNewsvendor(cu = cu, co = co_90, kernel_bandwidth = best_param_kbwth_90[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_90),2)
  result_gwn_90_avc.append(avc)

In [ ]:
GWN_separate_avc_90 = pd.DataFrame(result_gwn_90_avc, columns=['GWN_avc_90'])
GWN_separate_avc_90.index.name = 'item_id'

In [ ]:
result_gwn_90_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90, kernel_bandwidth = best_param_kbwth_90[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_90)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_90)
  result_gwn_90_pscr.append(pscr)

In [ ]:
GWN_separate_pscr_90 = pd.DataFrame(result_gwn_90_pscr, columns=['GWN_pscr_90'])
GWN_separate_pscr_90.index.name = 'item_id'

# gwn for 75% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,75
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_75 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_75.append(kbwth)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_gwn_75_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  
  knn = GaussianWeightedNewsvendor(cu = cu, co = co_75, kernel_bandwidth = best_param_kbwth_75[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_75),2)
  result_gwn_75_avc.append(avc)

In [ ]:
GWN_separate_avc_75 = pd.DataFrame(result_gwn_75_avc, columns=['GWN_avc_75'])
GWN_separate_avc_75.index.name = 'item_id'

In [ ]:
result_gwn_75_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75, kernel_bandwidth = best_param_kbwth_75[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_75)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_75)
  result_gwn_75_pscr.append(pscr)

In [ ]:
GWN_separate_pscr_75 = pd.DataFrame(result_gwn_75_pscr, columns=['GWN_pscr_75'])
GWN_separate_pscr_75.index.name = 'item_id'

# gwn for 50% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,5
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_kbwth_50 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50)

  random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  kbwth = random.best_params_.get('kernel_bandwidth')

  best_param_kbwth_50.append(kbwth)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_gwn_50_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  
  knn = GaussianWeightedNewsvendor(cu = cu, co = co_50, kernel_bandwidth = best_param_kbwth_50[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = gwn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_50),2)
  result_gwn_50_avc.append(avc)

In [ ]:
GWN_separate_avc_50 = pd.DataFrame(result_gwn_50_avc, columns=['GWN_avc_50'])
GWN_separate_avc_50.index.name = 'item_id'

In [ ]:
result_gwn_50_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  
  gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50, kernel_bandwidth = best_param_kbwth_50[prod])
  gwn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_gwn = gwn.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_50)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_gwn, preds_saa, cu = cu, co = co_50)
  result_gwn_50_pscr.append(pscr)

In [ ]:
GWN_separate_pscr_50 = pd.DataFrame(result_gwn_50_pscr, columns=['GWN_pscr_50'])
GWN_separate_pscr_50.index.name = 'item_id'

# Merging

In [ ]:
GWN_separated_avc = pd.concat([GWN_separate_avc_95, GWN_separate_avc_90, GWN_separate_avc_75, GWN_separate_avc_50], axis=1)
GWN_separated_avc

,GWN_avc_95,GWN_avc_90,GWN_avc_75,GWN_avc_50
item_id,,,,
0,2.63,2.85,3.60,5.60
1,1.31,1.73,2.75,4.59
2,4.65,5.19,6.57,10.15
3,13.63,14.81,18.86,29.73
4,5.57,6.20,7.92,12.69
5,2.33,2.86,3.92,6.85
6,2.71,3.08,4.19,6.93
7,5.58,6.13,7.69,11.42
8,1.51,1.89,2.84,4.82


In [ ]:
GWN_separated_pscr = pd.concat([GWN_separate_pscr_95, GWN_separate_pscr_90, GWN_separate_pscr_75, GWN_separate_pscr_50], axis=1)
GWN_separated_pscr

,GWN_pscr_95,GWN_pscr_90,GWN_pscr_75,GWN_pscr_50
item_id,,,,
0,0.385152,0.331200,0.268097,0.251664
1,0.309164,0.390164,0.400780,0.362367
2,0.231506,0.147689,0.141220,0.124378
3,0.524925,0.520132,0.574728,0.464470
4,0.348692,0.289382,0.179394,0.145331
5,0.269910,0.252949,0.279801,0.148366
6,0.485033,0.449844,0.442126,0.406690
7,0.411116,0.342492,0.226069,0.128594
8,0.267985,0.275786,0.231813,0.193591


Saving Files

In [ ]:
GWN_separated_avc.to_csv('/content/drive/MyDrive/M5/GWN_separated_avc.csv')

GWN_separated_pscr.to_csv('/content/drive/MyDrive/M5/GWN_separated_pscr.csv')